In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from natsort import natsorted
from glob import glob
from math import ceil
import hist
from hist import Hist
import mplhep as hep
plt.style.use(hep.style.CMS)
import boost_histogram as bh
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

# Set Defaults

In [ ]:
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_TOPBoard","ET2_BOTBoard","",""]
chip_fignames = chip_names
chip_figtitles = ["ETROC2 BB EPIR 1-1 HV210V OS:23","ETROC2 BB CNM 1-3 HV210V OS:23","",""]

chip_index= ["0","1","2","3"]

today = datetime.date.today().isoformat()
fig_outdir = Path('../ETROC-figures/DESYbeam')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

#NAME OF THE FOLDER WHERE THE DATA IS, check the date!
path_pattern = f"*{datetime.date.today().isoformat()}_Array_Test_Results/SelfTrigger_*_Readout_*_settings_*_loop_*"

In [ ]:
def return_hist():
    return {chip_name: hist.Hist(hist.axis.Regular(50, 140, 240, name="CAL", label="CAL [LSB]"), 
                          hist.axis.Regular(64, 0, 512,  name="TOT", label="TOT [LSB]"),
                          hist.axis.Regular(64, 0, 1024, name="TOA", label="TOA [LSB]"),
                          hist.axis.Regular(10, 0, 10,  name="numHits",label="Number of Hits in Event"),
                          hist.axis.Integer(0, 16, name="ROW", label="ROW"),
                          hist.axis.Integer(0, 16, name="COL", label="COL")
                          )
     for chip_name in chip_names}

In [ ]:
def fill_hist(chip_index, chip_name, hist, path_pattern=f"*{today}_Array_Test_Results/*", numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1)):
    root = '../ETROC-Data'
    #root = "../../../../run/media/daq/T7"
    file_pattern = "*translated*.dat"
    file_list = []
    totalNumHits = 0
    for path, subdirs, files in os.walk(root):
        if not fnmatch(path, path_pattern): continue
        for name in files:
            pass
            if fnmatch(name, file_pattern):
                file_list.append(os.path.join(path, name))
    print(file_list)
    for file_name in (file_list): print(file_name)
    for file_name in tqdm(file_list):
        with open(file_name) as infile:
            numHits = 0
            miniList = []
            for line in infile:
                text_list = line.split()
                if text_list[0] == "EH" or text_list[0] == "ET": continue
                if text_list[1]!=chip_index: continue
                if text_list[0]=="H":
                    writeList = True
                if text_list[0]=="T":
                    numHits = int(text_list[3])
                    if len(miniList)!=numHits: pass
                    elif(numHitsCut>-1 and numHits>numHitsCut): pass
                    else:
                        totalNumHits += numHits
                        for selectedLine in miniList:
                            text_mini_list = selectedLine.split()
                            TOA = int(text_mini_list[5])
                            if(TOAcut[0]>-1 and TOAcut[1]>-1 and (TOA<TOAcut[0] or TOA>=TOAcut[1])): continue
                            TOT = int(text_mini_list[6])
                            if(TOTcut[0]>-1 and TOTcut[1]>-1 and (TOT<TOTcut[0] or TOT>=TOTcut[1])): continue
                            CAL = int(text_mini_list[7])
                            if(CALcut[0]>-1 and CALcut[1]>-1 and (CAL<CALcut[0] or CAL>=CALcut[1])): continue
                            ROW = int(text_mini_list[4])
                            if(ROWcut[0]>-1 and ROWcut[1]>-1 and (ROW<ROWcut[0] or ROW>=ROWcut[1])): continue
                            COL = int(text_mini_list[3])
                            if(COLcut[0]>-1 and COLcut[1]>-1 and (COL<COLcut[0] or COL>=COLcut[1])): continue
                            hist[chip_name].fill(CAL,TOT,TOA,numHits,ROW,COL)
                        miniList  = []
                        writeList = False
                        numHits = 0
                # skip noisy pixel?
                # if(writeList and text_list[0]=="D"):
                #     if(int(text_list[4])==2 and int(text_list[3])==1 and text_list[1]=="0"): pass
                #     else: miniList.append(line)
    print("Total Number of Hits (After DF NHits cut)", totalNumHits)

In [ ]:
def make_inclusive_plots(chip_name, chip_figname, chip_figtitle, save=True, show=False, tag='', title_tag=''):
    
    fig = plt.figure(dpi=50, figsize=(20,20))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, Heat Map{title_tag}", loc="right", size=25)
    #hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("COL","ROW").plot2d(ax=ax, lw=1)
    ax.invert_xaxis()
    # ax.invert_yaxis()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_heatmap_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()
    
    # fig = plt.figure(dpi=50, figsize=(20,10))
    # gs = fig.add_gridspec(1,1)
    # ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"{chip_figtitle}, numHits{title_tag}", loc="right", size=25)
    # hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    # h[chip_name].project("numHits")[:10j].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_numHits_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20,10))
    # gs = fig.add_gridspec(1,1)
    # ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"{chip_figtitle}, CAL{title_tag}", loc="right", size=25)
    # hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    # h[chip_name].project("CAL")[:].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_CAL_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20,10))
    # gs = fig.add_gridspec(1,1)
    # ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"{chip_figtitle}, TOT{title_tag}", loc="right", size=25)
    # hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    # h[chip_name].project("TOT")[:].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOT_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20,10))
    # gs = fig.add_gridspec(1,1)
    # ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"{chip_figtitle}, TOA{title_tag}", loc="right", size=25)
    # hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    # h[chip_name].project("TOA")[:].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20,20))
    # gs = fig.add_gridspec(1,1)
    # ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"{chip_figtitle}, TOA v TOT{title_tag}", loc="right", size=25)
    # hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    # h[chip_name].project("TOA","TOT")[::2j,::2j].plot2d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_TOT_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20, 20))
    # h[chip_name].project("TOA","TOT")[::2j,::2j].plot2d_full(
    #     top_ls="-",
    #     top_color="orange",
    #     top_lw=1,
    #     side_ls="-",
    #     side_lw=1,
    #     side_color="steelblue",
    # )
    # plt.tight_layout()
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_TOT_full_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20, 20))
    # h[chip_name].project("TOA","CAL")[::2j,::1j].plot2d_full(
    #     top_ls="-",
    #     top_color="orange",
    #     top_lw=1,
    #     side_ls="-",
    #     side_lw=1,
    #     side_color="steelblue",
    # )
    # plt.tight_layout()
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_CAL_full_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

    # fig = plt.figure(dpi=50, figsize=(20, 20))
    # h[chip_name].project("TOT","CAL")[::2j,::1j].plot2d_full(
    #     top_ls="-",
    #     top_color="orange",
    #     top_lw=1,
    #     side_ls="-",
    #     side_lw=1,
    #     side_color="steelblue",
    # )
    # plt.tight_layout()
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOT_CAL_full_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    # if(show): plt.show()
    # plt.close()

In [ ]:
for idx in range(chip_index):
    h = return_hist()
    fill_hist(chip_index[idx], chip_names[idx], h, path_pattern=path_pattern, 
            numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1))
    make_inclusive_plots(chip_names[idx], chip_fignames[idx], chip_figtitles[idx], save=True, show=False, tag="tb_inc_", title_tag=", Inclusive")
    del h